In [1]:
import warnings

import pandas as pd
from google.cloud.bigquery import Client
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
     

In [2]:
categ_cols=['sla','product_type','brand','service_type','incident_type']
num_cols=['open_to_close_hour','response_to_resolved_hour']
label='severity_name'
unusedCols=['severity_id','severity_name']

target='severity'


In [6]:
df = pd.read_csv('../data/TrainEval_Incident.csv')
df

,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,severity
0,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,1.283333,1.000000,3
1,24x7 4Hrs Resolution Time,Server,HPE,Incident,General Incident,3.216667,3.000000,1
2,24x7 6Hrs Resolution Time,Software,VMWare,Request,General Incident,5.816667,5.816667,0
3,24x7 6Hrs Resolution Time,Software,Red Hat,Request,General Incident,96.450000,77.650000,0
4,24x7 4Hrs Response Time,Server,HPE,Incident,System Board Failure,19.100000,19.000000,2
...,...,...,...,...,...,...,...,...
2124,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,78.366667,0.200000,2
2125,24x7 6Hrs Response Time,Server,HPE,Incident,General Incident,239.783333,239.783333,2
2126,24x7 6Hrs Resolution Time,Software,SAPB1,Incident,Software,74.033333,72.083333,1
2127,24x7 4Hrs Response Time,Software,Trend Micro,Request,Software,2.000000,1.916667,1


In [5]:
df[target] = df[label].map({'Cosmatic':0,'Minor': 1, "Major": 2, "Critical": 3})

KeyError: 'severity_name'

In [ ]:
fig , ax = plt.subplots(figsize=(15,5))
ax =sns.countplot(x=label, data=df,)
for p in ax.patches:
   ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
plt.title(label.title())
plt.show()

In [162]:
sr_predict=df.iloc[-1,:]
df=df.iloc[0:len(df)-1,:]

listTarget=list(df[target].unique())
nTarget=len(listTarget)
print(f"TargetLabel :{listTarget } = {nTarget}")

df=df.drop(columns=unusedCols)

print(df.info())

TargetLabel :[0, 2, 1, 3] = 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sla                        2108 non-null   object 
 1   product_type               2108 non-null   object 
 2   brand                      2108 non-null   object 
 3   service_type               2108 non-null   object 
 4   incident_type              2108 non-null   object 
 5   open_to_close_hour         2108 non-null   float64
 6   response_to_resolved_hour  2108 non-null   float64
 7   severity                   2108 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 131.9+ KB
None


In [163]:
df.tail()

,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,severity
2103,24x7 4Hrs Resolution Time,Hardware,Oracle,Incident,Hard Disk Drive Failure,243.000000,238.916667,2
2104,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,General Incident,20.633333,20.566667,2
2105,24x7 4Hrs Response Time,Storage,EMC,Incident,General Incident,142.416667,125.633333,1
2106,24x7 4Hrs Response Time,Firewall,Palo Alto,Incident,Software,889.683333,888.966667,2
2107,24x7 4Hrs Response Time,Server,HPE,Incident,General Incident,3.500000,3.250000,2


In [164]:
def encode_cols(data, col):
    # Creating a dummy variable for the variable 'CategoryID' and dropping the first one.
    categ = pd.get_dummies(data[col], prefix=col, drop_first=True)
    # Adding the results to the master dataframe
    data = pd.concat([data, categ], axis=1)
    return data


# dummy-encode the categorical fields
for i in categ_cols:
    df = encode_cols(df, i)
    df.drop(columns=[i], inplace=True)

# check the data's shape
df.shape

(2108, 57)

In [165]:
X = df[[i for i in df.columns if i != target]].copy()
y = df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=36
)
print(X_train.shape, X_test.shape)
print(X_train.info() )
print(y_train.info())

(1686, 56) (422, 56)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1686 entries, 1114 to 645
Data columns (total 56 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   open_to_close_hour                     1686 non-null   float64
 1   response_to_resolved_hour              1686 non-null   float64
 2   sla_24x7 4Hrs Response Time            1686 non-null   uint8  
 3   sla_24x7 6Hrs Resolution Time          1686 non-null   uint8  
 4   sla_24x7 6Hrs Response Time            1686 non-null   uint8  
 5   sla_8x5 4Hrs Response Time             1686 non-null   uint8  
 6   sla_9x5xNBD 4Hrs Response Time         1686 non-null   uint8  
 7   product_type_Firewall                  1686 non-null   uint8  
 8   product_type_Hardware                  1686 non-null   uint8  
 9   product_type_Other                     1686 non-null   uint8  
 10  product_type_Security                  1686 non-n

In [166]:
scaler = StandardScaler()
X_train.loc[:, num_cols] = scaler.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = scaler.transform(X_test[num_cols])

In [167]:

y_train_categ = to_categorical(y_train)
y_test_categ = to_categorical(y_test)

In [168]:
optimizer = "adam"
num_hidden_layers = 2
num_neurons = [16, 16]
activ_func = ["relu", "relu"]


model = Sequential()
# construct the neural network as per the defined parameters
for i in range(num_hidden_layers):
    if i == 0:
        # add the input layer
        model.add(
            Dense(
                num_neurons[i],
                activation=activ_func[i],
                input_shape=(X_train.shape[1],),
            )
        )
    else:
        # add the hidden layers
        model.add(Dense(num_neurons[i], activation=activ_func[i]))

# add the output layer
model.add(Dense(nTarget, activation="softmax"))
# compile the model
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                912       
                                                                 
 dense_16 (Dense)            (None, 16)                272       
                                                                 
 dense_17 (Dense)            (None, 4)                 68        
                                                                 
Total params: 1,252
Trainable params: 1,252
Non-trainable params: 0
_________________________________________________________________


In [169]:
EPOCHS = 100
BATCH_SIZE = 32
history = model.fit(X_train, y_train_categ, epochs=EPOCHS,batch_size=BATCH_SIZE, verbose=1)

Epoch 1/100
53/53 [==============================] - 1s 1ms/step - loss: 1.2965
Epoch 2/100
53/53 [==============================] - 0s 1ms/step - loss: 1.1385
Epoch 3/100
53/53 [==============================] - 0s 1ms/step - loss: 1.0270
Epoch 4/100
53/53 [==============================] - 0s 1ms/step - loss: 0.9370
Epoch 5/100
53/53 [==============================] - 0s 1ms/step - loss: 0.8631
Epoch 6/100
53/53 [==============================] - 0s 1ms/step - loss: 0.8101
Epoch 7/100
53/53 [==============================] - 0s 1ms/step - loss: 0.7750
Epoch 8/100
53/53 [==============================] - 0s 1ms/step - loss: 0.7529
Epoch 9/100
53/53 [==============================] - 0s 1ms/step - loss: 0.7366
Epoch 10/100
53/53 [==============================] - 0s 2ms/step - loss: 0.7237
Epoch 11/100
53/53 [==============================] - 0s 1ms/step - loss: 0.7118
Epoch 12/100
53/53 [==============================] - 0s 1ms/step - loss: 0.7041
Epoch 13/100
53/53 [=================

In [170]:
test_results = model.evaluate(X_test, y_test_categ, verbose=1)
print(f"Test results - Loss: {test_results}")

14/14 [==============================] - 0s 1ms/step - loss: 0.7896
Test results - Loss: 0.789569079875946


In [ ]:
# GCS_PATH = BUCKET_URI + "/path-to-save/"
# model.save(GCS_PATH)